In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score, f1_score
import numpy as np

import matplotlib.pyplot as plt
import os
from sklearn.model_selection import cross_val_score
import json
import pickle as pkl
import utils

from cmath import exp
import dice_ml
from dice_ml import Dice

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split

from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier as rfcls


In [2]:
import importlib
importlib.reload(utils)

<module 'utils' from 'C:\\Users\\suhwan\\Desktop\\Project\\anomaly_detection_explainability\\utils.py'>

In [3]:
########################
### Data preparation ###
########################

df = pd.read_csv('./data/Prefix 5 dataset.csv')

used_models = 'RF'

key_pair = {'Case ID':'caseid', 'Activity':'activity', 'Complete Timestamp':'ts'}
df = df.rename(columns=key_pair)

if 'resource' in df.columns.values:
    df = df.loc[:,['caseid','activity','ts','resource','outcome','noise']]

else:
    df = df.loc[:,['caseid','activity','ts','noise', 'outcome']]

groups = df.groupby('caseid')
concating = []

outcome_dict = {x: pos+1 for pos, x in enumerate(list(set(df['outcome'])))}
new_outcome = []
for x in list(df['outcome']):
    new_outcome.append(outcome_dict[x])

df['outcome'] =new_outcome


outcome = []
for _, group in groups:
    group['ts'] = pd.to_datetime(group['ts'])
    group = group.sort_values(by='ts')
    group = group.reset_index(drop=True)
    case_length = len(group)
    prep_group = {'caseid': list(group['caseid'])[-1], 'outcome': list(group['outcome'])[-1]}
    
    ####################
    ###Activity label###
    ####################
    actlist = list(group['activity'])
    new_actcolumns =['Activity_%s'%(x+1) for x in range(case_length)]
    for pos, n in enumerate(new_actcolumns):
        prep_group[n] = actlist[pos]
    

    ####################
    ###  Timestamp   ###
    ####################

    durationlist = []
    for pos, x in enumerate(list(group['ts'])):
        if pos ==0:
            durationlist.append(0)
        else:
            durationlist.append((x - list(group['ts'])[pos-1]).total_seconds())
    duration_index ={'Duration_%s'%(x+1): durationlist[x] for x in range(len(durationlist))}
    cumdurationlist = [(x - list(group['ts'])[0]).total_seconds() for x in list(group['ts'])]
    cumduration_index ={'Cumduration_'+str(x+1): cumdurationlist[x] for x in range(len(cumdurationlist))}

    prep_group.update(duration_index)
    prep_group.update(cumduration_index)

    ####################
    ###   Resource   ###
    ####################
    
    if 'resource' in group.columns.values:
        reslist = list(group['resource'])
        new_resourceolumns = ['Resource_%s'%(x+1) for x in range(case_length)]
        for pos, n in enumerate(new_resourceolumns):
            prep_group[n] = reslist[pos]
    
    ####################
    ###  Concating   ###
    ####################
#     print(prep_group)
#     print(pd.DataFrame.from_dict(prep_group))

    ####################
    ###    Noise     ###
    ####################
    
    
    concating.append(prep_group)

dfn = pd.DataFrame.from_dict(concating)

datasetX = dfn.drop(columns=['caseid'],axis=1)

outcome_name = 'outcome'
categorical_features =[]
continuous_features = []


for x in datasetX.columns.values:
    if x =='caseid' or x==outcome_name:
        pass
    elif datasetX[x].dtype == 'object':
        categorical_features.append(x)
    else:
        continuous_features.append(x)

In [4]:
target = datasetX[outcome_name]
datasetXY = datasetX.drop(outcome_name,axis=1)
caseidset = set(dfn['caseid'])

trainidset = np.random.choice(list(caseidset), int(len(caseidset)*0.6), replace=False )
testidset = caseidset.difference(trainidset)

valididset = set(np.random.choice(list(testidset), int(len(testidset)*0.5), replace=False))
testidset = testidset.difference(valididset)

trainidset = sorted(list(trainidset))
valididset = sorted(list(valididset))
testidset = sorted(list(testidset))
print(len(trainidset), len(testidset), len(valididset))

x_train = datasetXY.iloc[trainidset,:]
y_train = target.iloc[trainidset]

x_valid = datasetXY.iloc[valididset, :]
y_valid = target.iloc[valididset]

x_test = datasetXY.iloc[testidset, :]
y_test = target.iloc[testidset]

# x_train, x_test, y_train, y_test = train_test_split(datasetXY,
#                                                     target,
#                                                     test_size=0.2,
#                                                     random_state=0,
#                                                     stratify=target)

3000 1000 1000


In [5]:
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

transformations = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, continuous_features),
        ('cat', categorical_transformer, categorical_features)])

In [6]:
# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
clf = Pipeline(steps=[('preprocessor', transformations),
                           ('classifier', rfcls(n_estimators = 10))])
model = clf.fit(x_train, y_train)

d = dice_ml.Data(dataframe=datasetX,
                  continuous_features=continuous_features,
                  outcome_name=outcome_name)

# We provide the type of model as a parameter (model_type)
m = dice_ml.Model(model=model, backend="sklearn", model_type='classifier')

exp_genetic = Dice(d, m, method="genetic")

In [7]:
valid_prediction = model.predict(x_valid)
print(classification_report(y_pred = valid_prediction, y_true = y_valid))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         9
           3       0.50      0.48      0.49       122
           4       0.00      0.00      0.00         5
           5       0.00      0.00      0.00         8
           6       0.00      0.00      0.00         5
           7       0.83      0.97      0.90       132
           8       0.00      0.00      0.00         6
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00         6
          11       0.86      0.99      0.92       259
          12       0.95      0.96      0.95       169
          13       0.30      0.26      0.28        82
          14       0.46      0.56      0.50       140
          15       0.00      0.00      0.00         7
          16       0.00      0.00      0.00         7
          17       0.95      0.76      0.84        25
          18       0.00    

c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\python3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [8]:
print(testidset[15])
print(x_test)

75
                                  Activity_1  \
14    start_event_Loan  application received   
23    start_event_Loan  application received   
25    start_event_Loan  application received   
30    start_event_Loan  application received   
32    start_event_Loan  application received   
...                                      ...   
4968  start_event_Loan  application received   
4969  start_event_Loan  application received   
4970    Return application back to applicant   
4981  start_event_Loan  application received   
4991  start_event_Loan  application received   

                                  Activity_2  \
14     Check  application  form completeness   
23     Check  application  form completeness   
25     Check  application  form completeness   
30     Check  application  form completeness   
32     Check  application  form completeness   
...                                      ...   
4968   Check  application  form completeness   
4969   Check  application  form comp

In [13]:
# Single input
query_instances = x_test[12:13]
prediction = model.predict(query_instances)[0]


actual_activity_label = [activity for activity,act_num in outcome_dict.items() if act_num == prediction][0]
wanted_activity_label = [activity for activity,act_num in outcome_dict.items() if act_num == 16][0]
print(actual_activity_label)
print(wanted_activity_label)
print(model.predict_proba(query_instances))
print(model.classes_)

features_to_vary =categorical_features + [x for x in continuous_features if 'Duration' in x]
features_to_vary.remove('Duration_1')
features_to_vary.remove('Activity_1')
print(features_to_vary)
genetic_iris = exp_genetic.generate_counterfactuals(query_instances, total_CFs=3, desired_class=2, features_to_vary=features_to_vary)
results = genetic_iris.visualize_as_dataframe(show_only_changes=False)
results

Return application back to applicant
start_event_Loan  application received
[[0.  0.  0.4 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.6 0.  0.  0.  0. ]]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18]
['Activity_2', 'Activity_3', 'Activity_4', 'Activity_5', 'Duration_2', 'Duration_3', 'Duration_4', 'Duration_5']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:01<00:00,  1.20s/it]

Query instance (original outcome : 14)


,Activity_1,Activity_2,Activity_3,Activity_4,Activity_5,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Cumduration_1,Cumduration_2,Cumduration_3,Cumduration_4,Cumduration_5,outcome
0,start_event_Loan application received,Check application form completeness,Return application back to applicant,Receive updated application,Check application form completeness,0.0,2360.302,1921.858,1204.576,7290.149,0.0,2360.302,4282.16,5486.736,12776.885,14



Diverse Counterfactual set (new outcome: 2.0)


,Activity_1,Activity_2,Activity_3,Activity_4,Activity_5,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Cumduration_1,Cumduration_2,Cumduration_3,Cumduration_4,Cumduration_5,outcome
0,start_event_Loan application received,Approve application,Return application back to applicant,Assess loan risk,Assess loan risk,0.0,364184.6,11462.8,26386.3,13067.0,0.0,2360.3,4282.16,5486.736,12776.885,2
0,start_event_Loan application received,Approve application,Return application back to applicant,end_event_Loan application rejected,Assess loan risk,0.0,364184.6,11462.8,26386.3,5896.7,0.0,2360.3,4282.16,5486.736,12776.885,2
0,start_event_Loan application received,Return application back to applicant,Send home insurance quote,Check credit history,Check application form completeness,0.0,7477.7,15936.2,8970.8,17100.4,0.0,2360.3,4282.16,5486.736,12776.885,2


In [15]:
print(genetic_iris.visualize_as_dataframe())

print(query_instances)
query_instances.to_csv('./testdf.csv',index=False)
genetic_iris.cf_examples_list[0].final_cfs_df_sparse.to_csv('./counterfactualdf.csv',index=False)

Query instance (original outcome : 14)


,Activity_1,Activity_2,Activity_3,Activity_4,Activity_5,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Cumduration_1,Cumduration_2,Cumduration_3,Cumduration_4,Cumduration_5,outcome
0,start_event_Loan application received,Check application form completeness,Return application back to applicant,Receive updated application,Check application form completeness,0.0,2360.302,1921.858,1204.576,7290.149,0.0,2360.302,4282.16,5486.736,12776.885,14



Diverse Counterfactual set (new outcome: 2.0)


,Activity_1,Activity_2,Activity_3,Activity_4,Activity_5,Duration_1,Duration_2,Duration_3,Duration_4,Duration_5,Cumduration_1,Cumduration_2,Cumduration_3,Cumduration_4,Cumduration_5,outcome
0,start_event_Loan application received,Approve application,Return application back to applicant,Assess loan risk,Assess loan risk,0.0,364184.6,11462.8,26386.3,13067.0,0.0,2360.3,4282.16,5486.736,12776.885,2
0,start_event_Loan application received,Approve application,Return application back to applicant,end_event_Loan application rejected,Assess loan risk,0.0,364184.6,11462.8,26386.3,5896.7,0.0,2360.3,4282.16,5486.736,12776.885,2
0,start_event_Loan application received,Return application back to applicant,Send home insurance quote,Check credit history,Check application form completeness,0.0,7477.7,15936.2,8970.8,17100.4,0.0,2360.3,4282.16,5486.736,12776.885,2


None
                                Activity_1  \
70  start_event_Loan  application received   

                               Activity_2  \
70  Check  application  form completeness   

                              Activity_3                   Activity_4  \
70  Return application back to applicant  Receive updated application   

                               Activity_5  Duration_1  Duration_2  Duration_3  \
70  Check  application  form completeness           0    2360.302    1921.858   

    Duration_4  Duration_5  Cumduration_1  Cumduration_2  Cumduration_3  \
70    1204.576    7290.149            0.0       2360.302        4282.16   

    Cumduration_4  Cumduration_5  
70       5486.736      12776.885  


In [9]:
np.random.seed(0)
caseids = list(set(datasetX['caseid']))
trainids = np.random.choice(caseids, int(len(caseids)*0.7), replace=False)
traindf = datasetX[datasetX['caseid'].isin(trainids)].reset_index(drop=True)
testdf = datasetX[~datasetX['caseid'].isin(trainids)].reset_index(drop=True)


y_train = traindf[outcome_name]
x_train = traindf.drop(columns=[outcome_name,'caseid'],axis=1)

y_test = testdf[outcome_name]
test_ids = set(testdf['caseid'])
x_test = testdf.drop(columns=[outcome_name,'caseid'],axis=1)


regr_housing = Pipeline(steps=[
                        ('classifier', rfcls())
                        ])

model_housing = regr_housing.fit(x_train, y_train)

datasetXy = datasetX.drop(columns=['caseid'], axis=1)

d_housing = dice_ml.Data(dataframe = datasetXy,
                        continuous_features = continuous_features,
                        outcome_name=outcome_name)
m_housing = dice_ml.Model(model=model_housing, backend='sklearn', model_type='classifier')

exp_genetic_housing = Dice(d_housing, m_housing, method='genetic')

test_df = datasetX[datasetX['caseid'].isin(test_ids)].sort_values(by='caseid')


In [ ]:
test_df = pd.read_csv('./testdf.csv')
query_instance_housing = test_df[3:4]

In [10]:
test_df = pd.read_csv('./testdf.csv')
query_instance_housing = test_df.iloc[3,:]
print(query_instance_housing.values)
test_outcome = query_instance_housing[outcome_name]
query_instance_housing = np.array(query_instance_housing.drop(labels=['caseid', outcome_name])).reshape(1,-1)
predicted_one = model_housing.predict(query_instance_housing)
model_classes = model_housing.classes_
predicted_proba = model_housing.predict_proba(query_instance_housing)
print(model_classes)
print(predicted_one, predicted_proba, test_outcome, list(model_classes).index(test_outcome))

[80 'Check if home insurance quote is requested' 0.0 88571.446 89439.822
 89439.822 90887.329 92987.689 93200.195 96452.673 96452.673 99664.339
 100398.908 102299.766 103284.663 111560.389 0.0 0.0 0.0 1.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0
 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 1.0
 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 1.0
 1.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0
 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
['Approve appli

In [12]:
query_instance_housing = test_df.iloc[3,:]
query_instance_housing = query_instance_housing.drop(labels=['caseid', outcome_name])
print(query_instance_housing)
genetic_housing = exp_genetic_housing.generate_counterfactuals(
                                query_instance_housing,
                                total_CFs=3,
                                desired_class='Reject application')
genetic_housing.visualize_as_dataframe(show_only_changes=True)



Cumduration_1                            0.0
Cumduration_2                      88571.446
Cumduration_3                      89439.822
Cumduration_4                      89439.822
Cumduration_5                      90887.329
                                     ...    
activity_3_Approve application           0.0
activity_9_Reject application            0.0
activity_5_Reject application            0.0
activity_6_Send acceptance pack          0.0
activity_6_Reject application            0.0
Name: 3, Length: 210, dtype: object


  0%|                                                                                                                                                             | 0/210 [00:00<?, ?it/s]


AttributeError: 'numpy.float64' object has no attribute 'columns'